In [1]:
#importing the libraries

import pandas as pd
import numpy as np
import datetime

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [3]:
dfs = []
for i in range(1,10):
    data = pd.read_parquet(f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-0{i}.parquet")
    dfs.append(data)

In [4]:
data = pd.DataFrame()

# Iterate over the list of DataFrames
for df in dfs:
    data = pd.concat([data, df], ignore_index=True)

In [5]:
data

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29328416,2,2022-09-30 23:46:31,2022-10-01 00:04:50,NaN,1.77,NaN,None,48,137,0,12.77,0.0,0.5,3.78,0.0,0.3,19.85,NaN,NaN
29328417,2,2022-09-30 23:43:29,2022-09-30 23:59:16,NaN,3.15,NaN,None,25,231,0,14.19,0.0,0.5,0.00,0.0,0.3,17.49,NaN,NaN
29328418,2,2022-09-30 23:45:30,2022-09-30 23:59:13,NaN,1.88,NaN,None,164,48,0,11.34,0.0,0.5,3.44,0.0,0.3,18.08,NaN,NaN
29328419,2,2022-09-30 23:09:37,2022-09-30 23:21:28,NaN,2.61,NaN,None,142,161,0,14.79,0.0,0.5,5.33,0.0,0.3,23.42,NaN,NaN


In [6]:
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [7]:
data.groupby('PULocationID')['VendorID'].count().sort_values()

PULocationID
199          9
105         10
84          12
99          14
27          15
        ...   
186     965979
161    1161686
236    1209130
237    1355797
132    1388223
Name: VendorID, Length: 262, dtype: int64

In [8]:
#lets use the 237 locationID since its the heavy volume place as per the above observation

data = data[data['PULocationID']==237]

In [9]:
data.shape

(1355797, 19)

The Idea is to predict the number of trips per day from Location 237

In [10]:
#taking the date and the number of trips per day

data['date'] = data['tpep_pickup_datetime'].dt.date

C:\Users\ManojKumar\AppData\Local\Temp\ipykernel_16196\2791146033.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data['tpep_pickup_datetime'].dt.date


In [11]:
Final_data = pd.DataFrame(data.groupby('date')['VendorID'].count())

In [12]:
Final_data.reset_index(inplace=True)

In [13]:
Final_data.columns = ['Date', 'No of trips']

In [14]:
#dropping the error dates

threshold_lower_date = pd.to_datetime('2022-01-01').date()
threshold_upper_date = pd.to_datetime('2022-09-30').date()

Final_data = Final_data[(Final_data['Date']>=threshold_lower_date)|(Final_data['Date']>=threshold_upper_date)]

In [15]:
Final_data

,Date,No of trips
9,2022-01-01,1764
10,2022-01-02,1837
11,2022-01-03,3425
12,2022-01-04,4002
13,2022-01-05,3974
...,...,...
279,2022-09-28,6504
280,2022-09-29,7089
281,2022-09-30,5966
282,2022-10-01,1


**Selecting the best window size**

In [ ]:
import dagshub
import mlflow

dagshub.init("MLOPS_Project_2023", "manojkotary", mlflow=True)
mlflow.set_experiment('Selecting_best_window_size')

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()

In [41]:
for window_size in range(1,20):
    with mlflow.start_run():
        # Create empty lists to store the input and output data
        X = []
        y = []
        for i in range(window_size, len(Final_data)):
            # Extract the 12-day sequence as input
            input_sequence = Final_data['No of trips'].iloc[i-window_size:i].tolist()
            
            # Extract the 13th day count as output
            output_value = Final_data['No of trips'].iloc[i]
            
            # Append the input and output to the respective lists
            X.append(input_sequence)
            y.append(output_value)

        # Create a new DataFrame from the input and output lists
        X_df = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(window_size)])
        y_df = pd.DataFrame({'y': y})
        new_df = pd.concat([X_df, y_df], axis=1)


        X = new_df.iloc[:, :-1]
        y = new_df['y']

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Convert the data into DMatrix format for XGBoost
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)

        # Set the parameters for the XGBoost model
        params = {
            'objective': 'reg:squarederror',  # Use squared error as the objective for regression
            'eval_metric': 'rmse'  # Root Mean Squared Error (RMSE) as the evaluation metric
        }

        # Train the XGBoost model
        num_rounds = 100  # Number of boosting rounds
        model = xgb.train(params, dtrain, num_rounds)

        # Make predictions on the test set
        y_pred = model.predict(dtest)

        # Calculate RMSE on the test set
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metric('Window_size', f"{window_size}")
        mlflow.log_metric("rmse", rmse)

**Retriving the best window size**

In [42]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "https://dagshub.com/manojkotary/MLOPS_Project_2023.mlflow"

In [48]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

#to get the list of experiments
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/9d850c2f83474fdbbdd8f377de7994d7', creation_time=1688361939657, experiment_id='2', last_update_time=1688361939657, lifecycle_stage='active', name='Selecting_best_window_size', tags={}>]

In [51]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='2',
    order_by=["metrics.rmse ASC"]
)

In [1]:
for run in runs:
    print(f"run id: {run.info.run_id}, window_size: {int(run.data.metrics['Window_size'])}, rmse: {run.data.metrics['rmse']:.4f}")

NameError: name 'runs' is not defined

**Building model with window length of 7**

In [59]:
# Create empty lists to store the input and output data
X = []
y = []
for i in range(7, len(Final_data)):
    # Extract the 7-day sequence as input
    input_sequence = Final_data['No of trips'].iloc[i-7:i].tolist()
    
    # Extract the 8th day count as output
    output_value = Final_data['No of trips'].iloc[i]
    
    # Append the input and output to the respective lists
    X.append(input_sequence)
    y.append(output_value)

# Create a new DataFrame from the input and output lists
X_df = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(7)])
y_df = pd.DataFrame({'y': y})
new_df = pd.concat([X_df, y_df], axis=1)


X = new_df.iloc[:, :-1]
y = new_df['y']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data into DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set the parameters for the XGBoost model
params = {
    'objective': 'reg:squarederror',  # Use squared error as the objective for regression
    'eval_metric': 'rmse'  # Root Mean Squared Error (RMSE) as the evaluation metric
}

# Train the XGBoost model
num_rounds = 100  # Number of boosting rounds
model = xgb.train(params, dtrain, num_rounds)

# Make predictions on the test set
y_pred = model.predict(dtest)

# Calculate RMSE on the test set
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

703.2583412272791

**Selecting best param for window size=7**

In [ ]:
# Create empty lists to store the input and output data
X = []
y = []
for i in range(7, len(Final_data)):
    # Extract the 7-day sequence as input
    input_sequence = Final_data['No of trips'].iloc[i-7:i].tolist()
    
    # Extract the 8th day count as output
    output_value = Final_data['No of trips'].iloc[i]
    
    # Append the input and output to the respective lists
    X.append(input_sequence)
    y.append(output_value)

# Create a new DataFrame from the input and output lists
X_df = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(7)])
y_df = pd.DataFrame({'y': y})
new_df = pd.concat([X_df, y_df], axis=1)


X = new_df.iloc[:, :-1]
y = new_df['y']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert the data into DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [60]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [61]:
mlflow.set_experiment('Selecting_best_parameters')

MlflowException: API request to endpoint /api/2.0/mlflow/experiments/get-by-name failed with error code 401 != 200. Response body: '=============== ATTENTION! ===============

Your access token has expired. To procees, you must first: 
    Use the DagsHub API to generate a new token 
    OR     Manually create one in: https://dagshub.com/user/settings/tokens
=========================================='

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=10,
            evals=[(dtest, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(dtest)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)